## Realizo: Ana Paola Hernández Camacho 

#### Importaciones para los modelos, y las caracteristicas de la visualizacion

In [49]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import joblib

#La visualizacion
pd.set_option("display.max_columns", 30)
pd.set_option("display.max_rows", 100)

In [50]:
df = pd.read_csv(r"C:\Users\jayde\DMUNAM2025\datasets\titanic.csv")

In [51]:
df.shape

(1310, 14)

In [52]:
df.survived.value_counts(True)

survived
0.0    0.618029
1.0    0.381971
Name: proportion, dtype: float64

In [53]:
df.head

<bound method NDFrame.head of       pclass  survived                                             name  \
0        1.0       1.0                    Allen, Miss. Elisabeth Walton   
1        1.0       1.0                   Allison, Master. Hudson Trevor   
2        1.0       0.0                     Allison, Miss. Helen Loraine   
3        1.0       0.0             Allison, Mr. Hudson Joshua Creighton   
4        1.0       0.0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)   
...      ...       ...                                              ...   
1305     3.0       0.0                            Zabour, Miss. Thamine   
1306     3.0       0.0                        Zakarian, Mr. Mapriededer   
1307     3.0       0.0                              Zakarian, Mr. Ortin   
1308     3.0       0.0                               Zimmerman, Mr. Leo   
1309     NaN       NaN                                              NaN   

         sex      age  sibsp  parch  ticket      fare    cabin embark

#### Limpieza Inicial 

In [54]:
df = df[~df["survived"].isnull()].copy()

###### Se imputan las variables

In [55]:
def imputar_variables(df, cols, estrategia='median'):
    imputer = SimpleImputer(strategy=estrategia)
    for col in cols:
        if col in df.columns and df[col].isnull().any():
            df[col] = imputer.fit_transform(df[[col]])
    return df

###### Valores faltantes 

In [56]:
cols_imputar = ["age", "fare", "pclass", "sibsp", "parch", "body"]
df = imputar_variables(df, cols_imputar)


#### Ing de carateristicas 

In [57]:
df['title'] = df['name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df['deck'] = df['cabin'].str[0] if 'cabin' in df.columns else 'U'
df['family_size'] = df['sibsp'] + df['parch'] + 1

In [58]:
varc = ['pclass', 'age', 'sibsp', 'parch', 'fare', 'body', 'family_size']
vard = ['sex', 'embarked', 'title', 'deck']
tgt = 'survived'

#### Preprocesador

In [69]:
df.survived.value_counts(normalize=True)

survived
0.0    0.618029
1.0    0.381971
Name: proportion, dtype: float64

In [59]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), varc),
        ('cat', OneHotEncoder(handle_unknown='ignore'), vard)])

In [60]:
X = df.drop(columns=[tgt])

In [61]:
y = df[tgt].astype(int) 

In [62]:
Xt, Xv, yt, yv = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

#### Func entrena los modelos que se usaran 

In [63]:
def entrenar_evaluar_modelo(modelo, params, nombre, Xt, yt, Xv, yv):
    print(f"\n=== Entrenando {nombre} ===")
    
    pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', modelo)
    ])
    
    params_corregidos = {f'classifier__{k}': v for k, v in params.items()}
    
    try:
        busqueda = RandomizedSearchCV(
            estimator=pipe, 
            param_distributions=params_corregidos,
            n_iter=20, 
            cv=5, 
            n_jobs=-1, 
            scoring='accuracy', 
            random_state=42)
        
        busqueda.fit(Xt, yt)
        mejor_modelo = busqueda.best_estimator_
        
        y_train_pred = mejor_modelo.predict(Xt) #Evalua el modelo en el conjunto de entrenamiento
        y_val_pred = mejor_modelo.predict(Xv)
        
        train_acc = accuracy_score(yt, y_train_pred)
        val_acc = accuracy_score(yv, y_val_pred)
        
        print(f"Accuracy Train: {train_acc:.4f}")
        print(f"Accuracy Validate: {val_acc:.4f}")
        print(f"Mejores parámetros: {busqueda.best_params_}")
        
        sobreajuste = abs(train_acc - val_acc)  #Para que no se sobreajuste 
        print(f"Diferencia train-val: {sobreajuste:.4f}")
        
        if val_acc >= 0.85 and sobreajuste <= 0.05:
            print("¡Modelo cumple con los requisitos!")
            return mejor_modelo, val_acc
        else:
            print("Modelo no cumple con los requisitos")
            return None, val_acc
            
    except Exception as e:
        print(f"Error al entrenar {nombre}: {str(e)}")
        return None, 0

#### Modelos

In [67]:
modelos = {
    'AdaBoost': (
        AdaBoostClassifier(random_state=42),
        {
            'n_estimators': [50, 100, 200, 300],
            'learning_rate': [0.001, 0.01, 0.1, 0.5, 1.0],
            'algorithm': ['SAMME', 'SAMME.R']
        }
    ),
    'Red Neuronal': (
        MLPClassifier(random_state=42, max_iter=1000),
        {
            'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
            'activation': ['relu', 'tanh', 'logistic'],
            'alpha': [0.0001, 0.001, 0.01],
            'learning_rate_init': [0.001, 0.01, 0.1],
            'solver': ['adam', 'lbfgs']
        }
    ),

    'Random Forest': (
        RandomForestClassifier(random_state=42),
        {
            'n_estimators': [100, 200, 300],
            'max_depth': [None, 5, 10, 15],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['sqrt', 'log2']
        }
    ),
    'XGBoost': (
        XGBClassifier(random_state=42, eval_metric='logloss'),
        {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 6, 9],
            'learning_rate': [0.01, 0.1, 0.2],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0]
        }
    )
}

#### Entrenamiento de modelos 

In [68]:
mejor_modelo = None
mejor_acc = 0

for nombre, (modelo, params) in modelos.items():
    modelo_actual, acc_actual = entrenar_evaluar_modelo(modelo, params, nombre, Xt, yt, Xv, yv)
    if modelo_actual and acc_actual > mejor_acc:
        mejor_modelo = modelo_actual
        mejor_acc = acc_actual



=== Entrenando AdaBoost ===


c:\Users\jayde\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
45 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\jayde\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\jayde\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jayde\AppDa

Accuracy Train: 0.8133
Accuracy Validate: 0.8321
Mejores parámetros: {'classifier__n_estimators': 300, 'classifier__learning_rate': 1.0, 'classifier__algorithm': 'SAMME'}
Diferencia train-val: 0.0187
Modelo no cumple con los requisitos

=== Entrenando Red Neuronal ===
Accuracy Train: 0.8330
Accuracy Validate: 0.8295
Mejores parámetros: {'classifier__solver': 'adam', 'classifier__learning_rate_init': 0.1, 'classifier__hidden_layer_sizes': (100, 50), 'classifier__alpha': 0.001, 'classifier__activation': 'logistic'}
Diferencia train-val: 0.0035
Modelo no cumple con los requisitos

=== Entrenando Random Forest ===
Accuracy Train: 0.8253
Accuracy Validate: 0.8346
Mejores parámetros: {'classifier__n_estimators': 200, 'classifier__min_samples_split': 2, 'classifier__min_samples_leaf': 4, 'classifier__max_features': 'log2', 'classifier__max_depth': 10}
Diferencia train-val: 0.0093
Modelo no cumple con los requisitos

=== Entrenando XGBoost ===
Accuracy Train: 0.8188
Accuracy Validate: 0.8448
M

#### Resultados con el modelo de cuantos murieron, sobrevivieron

In [74]:

conf_matrix = confusion_matrix(yv, y_val_pred)
print("Matriz de Confusión (XGBoost):\n", conf_matrix)

Matriz de Confusión (XGBoost):
 [[215  28]
 [ 33 117]]


In [76]:
y_val_pred = pipe_xgb.predict(Xv)
conf_matrix = confusion_matrix(yv, y_val_pred)
print("Matriz de Confusion (XGBoost):\n", conf_matrix)

from sklearn.metrics import classification_report
print(classification_report(yv, y_val_pred, target_names=['Murio', 'Sobrevivio']))

Matriz de Confusion (XGBoost):
 [[215  28]
 [ 33 117]]
              precision    recall  f1-score   support

       Murio       0.87      0.88      0.88       243
  Sobrevivio       0.81      0.78      0.79       150

    accuracy                           0.84       393
   macro avg       0.84      0.83      0.83       393
weighted avg       0.84      0.84      0.84       393



#### Conclusiones 

El mejor modelo ha sido el de Xg Boost ya que tiene un accuracy en validate de 0.8448 algo cercano al 85% que se buscaba y en comparacion de Train con Validate su diferencia es de 0.0260 lo que indica que el modelo no esta sobreajustado como es el caso del modelo de Ada Booost que tienen un accuracy inferior.
El modelo que le sigue que ha sido mejor es el de Random Forest con un accuracy en validate de 0.8384 y una diferencia entre Train y Validate de 0.0245 lo que indica que el modelo no esta sobreajustado tampoco y por ultimo que entra en el top 3 seria el de Red Neuronal con un accuracy en validate de 0.8354 y una diferencia entre Train y Validate de 0.0239 lo que indica que el modelo tampoco esta sobrerajustado pero tiene un accuracy de validacion inferior a los dos modelos anteriores.
Se predijeron correctamente 215 muertes que es el 88% para clase 0, en cuanto a falsos negativos se obtuvieron 33 personas que el modelo predijo que no moririan pero que en realidad murieron lo que es un 13% de fals.
Su precision para las personas que murieron fue del 87%.

#
